In [4]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset


df = pd.read_csv('/kaggle/input/drcat-dataset/train_v2_drcat_02.csv')
texts = df['text'].tolist()  
labels = df['label'].tolist()  

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)


tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')
    precision = precision_score(labels, predictions, average='binary')
    recall = recall_score(labels, predictions, average='binary')
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


class EssaysDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

    
train_dataset = EssaysDataset(train_encodings, train_labels)
val_dataset = EssaysDataset(val_encodings, val_labels)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)



training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.02,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.000200,0.019197,0.996434,0.995408,0.997986,0.992843
2,0.000100,0.010473,0.997326,0.996564,0.996849,0.996278


TrainOutput(global_step=4488, training_loss=0.03980766757551283, metrics={'train_runtime': 2103.7168, 'train_samples_per_second': 34.124, 'train_steps_per_second': 2.133, 'total_flos': 9509569614716928.0, 'train_loss': 0.03980766757551283, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()
test_df = pd.read_csv('/kaggle/input/textdata/test.csv')
test_texts = test_df['text'].tolist()

test_encodings = tokenizer(test_texts, truncation=True, padding=True)

class TestDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

test_dataset = TestDataset(test_encodings)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) 

# Vorhersagen treffen
model.eval()
predictions = []
with torch.no_grad():

    for item in test_dataset:
        inputs = {k: v.to(device).unsqueeze(0) for k, v in item.items()}
        output = model(**inputs)
        #logits = output.logits
        
        
    
        logits = output.logits
        predicted_class_id = torch.sigmoid(logits)
        predicted_class_id = accelerator.gather_for_metrics(predicted_class_id)
        predicted_class_id = predicted_class_id.cpu().numpy().tolist()
    
        predictions.append(predicted_class_id)

test_df['generated'] = predictions
test_df["generated"] = test_df["generated"].rank(method='min')
test_df.sort_values(by='id').head()
test_df.groupby("id")["generated"].mean().reset_index()
test_df.to_csv('test_predictions7.csv', index=False)
print(test_df.head())